In [26]:
!pip install map_boxes
!pip install tqdm
!pip install pycocotools

In [36]:
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from pycocotools.coco import COCO

In [37]:
# Prediction

PRED_CSV = '../../sample_submission/train_sample.csv'

pred_df = pd.read_csv(PRED_CSV)

print(type(pred_df))

pred_df

<class 'pandas.core.frame.DataFrame'>


,PredictionString,image_id
0,0 0.7978816628456116 183.583740234375 204.7623...,train/0000.jpg
1,1 0.6951920390129089 125.23683166503906 0.0 10...,train/0001.jpg
2,3 0.764673113822937 262.1150817871094 169.8388...,train/0002.jpg
3,0 0.8639018535614014 764.7205200195312 16.2596...,train/0003.jpg
4,1 0.919501543045044 485.4276428222656 483.8736...,train/0004.jpg
...,...,...
4878,0 0.48651012778282166 382.0939636230469 369.53...,train/4878.jpg
4879,4 0.7255597710609436 158.5552520751953 446.560...,train/4879.jpg
4880,0 0.8047160506248474 189.45559692382812 215.36...,train/4880.jpg
4881,7 0.9831897020339966 146.3490753173828 477.500...,train/4881.jpg


In [38]:
file_names = pred_df['image_id'].values.tolist()
bboxes = pred_df['PredictionString'].values.tolist()

print(file_names[0])
print(bboxes[0])

train/0000.jpg
0 0.7978816628456116 183.583740234375 204.7623748779297 757.228759765625 663.2476196289062 0 0.30007776618003845 294.1588134765625 428.72027587890625 712.8113403320312 662.5757446289062 9 0.27576687932014465 187.20274353027344 199.62417602539062 785.662109375 664.3910522460938 0 0.13876120746135712 360.568359375 198.63479614257812 678.2484741210938 474.7105712890625 1 0.09028089046478271 222.17236328125 184.7045135498047 727.7378540039062 673.251708984375 9 0.07502871006727219 363.07427978515625 200.3661651611328 688.6143798828125 481.45367431640625 0 0.07456113398075104 203.72471618652344 370.8855285644531 483.5150146484375 544.1288452148438 


In [39]:
# check if there's any empty box
for i, bbox in enumerate(bboxes):
    if isinstance(bbox, float): # bbox type is str
        print(f'{file_names[i]} empty box')

In [40]:
new_pred = []

for file_name, bbox in tqdm(zip(file_names, bboxes)):
    boxes = np.array(str(bbox).split(' '))
    # print(boxes)
    
    if len(boxes) % 6 == 1: # some lines may have an empty str '' as the last item.
        boxes = boxes[:-1].reshape(-1, 6)
        # print('%6==1\n', boxes)
    elif len(boxes) % 6 == 0:
        boxes = boxes.reshape(-1, 6)
        # print('%6==0\n', boxes)
    else:
        raise Exception('error', 'invalid box count')
        
    for box in boxes:
        new_pred.append([file_name, box[0], box[1], float(box[2]), float(box[4]), float(box[3]), float(box[5])])
        # class, confidence, four coordinates
        
print(len(new_pred))
print(new_pred[0])

4883it [00:01, 4575.77it/s]

151039
['train/0000.jpg', '0', '0.7978816628456116', 183.583740234375, 757.228759765625, 204.7623748779297, 663.2476196289062]


In [41]:
# Ground Truth

GT_JSON = '../../dataset/train.json'

coco = COCO(GT_JSON)
   
'''
coco.getImgIds(): return image id list
    
coco.loadImgs(image_id): return image_info
    
coco.getAnnIds(imgIds=image_info['id']): return annotation id
    
coco.loadAnns(ann_ids): return annotation information list (annotation_info_list)
    
image_info['file_name']: return file name

annotation_info_list[i]['category_id']: return i'th annotation category

annotation_info_list[i]['bbox']: return i'th annotation [x_min, y_min, w, h]    
'''

gt = []

for image_id in coco.getImgIds():
        
    image_info = coco.loadImgs(image_id)[0]
    annotation_id = coco.getAnnIds(imgIds=image_info['id'])
    annotation_info_list = coco.loadAnns(annotation_id)
        
    file_name = image_info['file_name']
        
    for annotation in annotation_info_list:
        gt.append([file_name, annotation['category_id'],
                   float(annotation['bbox'][0]),
                   float(annotation['bbox'][0]) + float(annotation['bbox'][2]),
                   float(annotation['bbox'][1]),
                   (float(annotation['bbox'][1]) + float(annotation['bbox'][3]))])

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!


In [42]:
mean_ap, average_precisions = mean_average_precision_for_boxes(gt, new_pred, iou_threshold=0.5)

print(mean_ap)

Number of files in annotations: 4883
Number of files in predictions: 4883
Unique classes: 10
Detections length: 4883
Annotations length: 4883
0                              | 0.264624 |    3966
1                              | 0.522890 |    6352
2                              | 0.441265 |     897
3                              | 0.411529 |     936
4                              | 0.604017 |     982
5                              | 0.397688 |    2943
6                              | 0.490678 |    1263
7                              | 0.722987 |    5178
8                              | 0.599785 |     159
9                              | 0.531735 |     468
mAP: 0.498720
0.49871974228579613
